# Introduction

In [ ]:
%run "./Getting started.ipynb"
import requests
base_url = 'http://localhost:6000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

We can run successive optimizations; for example, let's run a two-stage grid search, starting with a coarse search to acquire signal followed by a finer search. To finish, we'll fit a Gaussian model to the results.

In [ ]:
import numpy as np
from emergent.pipeline import Pipeline, Source, GridSearch, GaussianModel, Rescale 
            
''' Define data source '''
hub = network.hubs['hub']
thing = hub.children['thing']
state = {'thing': thing.state}
bounds = hub.range.copy()
experiment = hub.gaussian
params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0.0}
source = Source(state, bounds, experiment, params)

''' Declare and run pipeline.  '''
pipe = Pipeline(state, source)

pipe.add(GridSearch(params={'Steps': 5}))
pipe.add(Rescale(threshold=0.5))
pipe.add(GridSearch(params={'Steps':30}))
pipe.add(GaussianModel(optimizer = GridSearch(params={'Steps': 30})))

points, costs = pipe.run()

print(points[-1], costs[-1])
print('done')
